In [18]:
import pymysql
import sys
from config.DatabaseConfig import *

In [19]:
db = None
try:
    db = pymysql.connect(host=DB_HOST, user=DB_USER, password=DB_PASSWORD, db=DB_NAME, charset='utf8')
    cursor = db.cursor()
    cursor.execute("SELECT VERSION()")  
    data = cursor.fetchone()
    print("Database version : %s " % data)
    print("Database connected successfully")
    
except Exception as e:
    print("Exception occured:{}".format(e))

Database version : 8.0.35 
Database connected successfully


In [20]:
def create_table():
    try:
        sql = '''
        CREATE TABLE IF NOT EXISTS `train` (
            id INT UNSIGNED NOT NULL AUTO_INCREMENT,
            intent VARCHAR(45) NULL,
            ner VARCHAR(1024) NULL,
            query TEXT NULL,
            answer TEXT NOT NULL,
            answer_image VARCHAR(2048) NULL,
            PRIMARY KEY (id)
        )
        '''
        
        with db.cursor() as cursor:
            cursor.execute(sql)
    
    except Exception as e:
        print("Exception occured:{}".format(e))

In [21]:
create_table()

In [22]:
def all_clear_train_data(db):
    # 기존 학습 데이터 삭제
    sql = '''
            delete from train
        '''
    with db.cursor() as cursor:
        cursor.execute(sql)

    # auto increment 초기화
    sql = '''
    ALTER TABLE train AUTO_INCREMENT=1
    '''
    with db.cursor() as cursor:
        cursor.execute(sql)

In [23]:
all_clear_train_data(db)

In [24]:
# db에 데이터 저장
def insert_data(db, xls_row):
    intent, ner, query, answer, answer_img_url = xls_row

    sql = '''
        INSERT train(intent, ner, query, answer, answer_image)
        values(
         '%s', '%s', '%s', '%s', '%s'
        )
    ''' % (intent.value, ner.value, query.value, answer.value, answer_img_url.value)

    # 엑셀에서 불러온 cell에 데이터가 없는 경우, null 로 치환
    sql = sql.replace("'None'", "null")

    with db.cursor() as cursor:
        cursor.execute(sql)
        print('{} 저장'.format(query.value))
        db.commit()

In [25]:
import openpyxl
train_file = './train_data.xlsx'
db = None
try:
    db = pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        passwd=DB_PASSWORD,
        db=DB_NAME,
        charset='utf8'
    )

    # 기존 학습 데이터 초기화
    all_clear_train_data(db)

    # 학습 엑셀 파일 불러오기
    wb = openpyxl.load_workbook(train_file)
    sheet = wb['Sheet1']
    for row in sheet.iter_rows(min_row=2): # 해더는 불러오지 않음
        # 데이터 저장
        insert_data(db, row)

    wb.close()

except Exception as e:
    print(e)

finally:
    if db is not None:
        db.close()

안녕하세요 저장
반가워요 저장
{B_FOOD} 주문할게요 저장
{B_FOOD} 주문할게요 저장
{B_DT} 예약 저장
None 저장
